In [1]:
#export
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

## Very good reading on inizialition! 

https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79
https://pouannes.github.io/blog/initialization/?fbclid=IwAR0lqZFCUUedF5edQS8z-RdsfqmI7fth-gk6ePIvHa2SChHNX6IVvr_oeAM




In [2]:
def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"
def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)

def test_near(a,b): 
    test(a,b,near)

In [3]:
def get_data(): 
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _)= pickle.load(f,encoding='latin-1')
    return map(tensor,(x_train, y_train ,x_valid, y_valid))


def normalize(x, m, s): 
    return (x-m)/s


In [4]:
x_train,y_train,x_valid,y_valid = get_data()


In [5]:
train_mean,train_std = x_train.mean(), x_train.std() 

In [6]:
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [7]:
x_train = normalize(x_train, train_mean, train_std)
# Important here is that we use the train_mean and train_std instead of the validation
x_valid = normalize(x_valid, train_mean, train_std)


In [8]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.0001), tensor(1.))

In [9]:
assert train_mean<0.01

In [10]:
n,m = x_train.shape
c = y_train.max()+1

In [11]:
n,m,c

(50000, 784, tensor(10))

In [12]:

#export
def test_near_zero(a,tol=1e-3): 
    assert a.abs()<tol, f"Near zero: {a}"

5000 rows, with 784 long tensors 28,28 images and 10 diffrent classes! 

## Foundations vesrison 

### Basic architecture

# https://google-developers.appspot.com/machine-learning/crash-course/backprop-scroll/

In [13]:
# nh nbr of hidden units 

nh = 50 

In [14]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [15]:
w1.shape

torch.Size([784, 50])

In [16]:
w1.std()

tensor(0.0358)

In [17]:
w1.mean()

tensor(0.0003)

In [18]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [19]:
x_valid.mean(),x_valid.std()


(tensor(-0.0057), tensor(0.9924))

In [20]:
def lin(x,w,b): 
    return x@w+b

In [21]:
t  = lin(x_valid, w1, b1)

In [22]:
t.shape

torch.Size([10000, 50])

In [23]:
t.mean(),t.std()

(tensor(-0.0419), tensor(0.9366))

In [24]:
def relu(x):
    return  x.clamp_min(0.)

In [25]:
t = relu(lin(x_valid, w1, b1))

In [26]:
t.mean(), t.std()

(tensor(0.3492), tensor(0.5441))

# pytorch text docs! 

In [27]:
w1 = torch.randn(m,nh)*math.sqrt(2/m)


In [28]:
w1.mean(),w1.std()


(tensor(0.0001), tensor(0.0506))

In [29]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.6182), tensor(0.8824))

In [30]:
#export
from torch.nn import init


In [31]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()


(tensor(0.5758), tensor(0.8025))

In [32]:
#init.kaiming_normal_??

In [33]:
w1.mean(),w1.std()

(tensor(2.7116e-05), tensor(0.0506))

In [34]:
t.mean(),t.std()


(tensor(0.5758), tensor(0.8025))

In [35]:
w1.shape

torch.Size([784, 50])

In [36]:
import torch.nn

In [37]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [38]:
torch.nn.Linear.forward??

Signature: torch.nn.Linear.forward(self, input)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    @weak_script_method
    def forward(self, input):
        return F.linear(input, self.weight, self.bias)
File:      ~/anaconda3/lib/python3.6/site-packages/torch/nn/modules/linear.py
Type:      function


In [39]:
torch.nn.functional.linear??

Signature: torch.nn.functional.linear(input, weight, bias=None)
Source:   
@weak_script
def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

    Shape:

        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())
        if bias is not None:
            output += bias
        ret = output
    return ret
File:      ~/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py
Type:      function


In [40]:
#torch.nn.Conv2d??

In [41]:
def relu(x): 
    return x.clamp_min(0.)-0.5

In [42]:
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0543), tensor(0.8521))

In [43]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [44]:
%timeit -n 10 _=model(x_valid)
out1=model(x_valid)


9.2 ms ± 933 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
out1.shape

torch.Size([10000, 1])

## Loss Functions: MSE 

In [46]:
model(x_valid).shape

torch.Size([10000, 1])

In [47]:
def mse(output,targ): 
    return (output.squeeze(-1) - targ).pow(2).mean()

In [48]:
y_train,y_valid = y_train.float(),y_valid.float()


In [49]:
preds = model(x_train)


In [50]:
preds.squeeze(-1)

tensor([ 0.3427,  0.4786,  0.0393,  ...,  0.4528, -0.4609,  0.3474])

In [51]:
y_train[0]

tensor(5.)

In [52]:
preds

tensor([[ 0.3427],
        [ 0.4786],
        [ 0.0393],
        ...,
        [ 0.4528],
        [-0.4609],
        [ 0.3474]])

In [53]:
mse(preds, y_train)


tensor(29.3088)

In [54]:
y_train

tensor([5., 0., 4.,  ..., 8., 4., 8.])

## Gradiets and backwards pass



### KEDJE REGLEN HELA VÄGEN! 

https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/

In [75]:
def mse_grad(inp,targ): 
    inp.g = 2. * (inp.squeeze()-targ).unsqueeze(-1)/inp.shape[0] 
    
# OK 

In [76]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    # inp > 0 means if tha value is bigger then the value we check it get's a 1. Otherwise is zero
    inp.g = (inp>0).float() * out.g
    
#derivitive of a ax+b conserning x is a
# OK

In [77]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t() # net in niklas example 
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0) 
    b.g = out.g.sum(0)

In [78]:
from IPython import embed
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    l3 = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(l3, targ)
    # backward pass:
    mse_grad(l3, targ)
    lin_grad(l2, l3, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [79]:
forward_and_backward(x_train, y_train)

tensor([[1., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        [0., 1., 0.,  ..., 1., 1., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 1.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 1., 0.]])


In [60]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [61]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [62]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [63]:
loss = forward(xt2, y_train)

In [64]:
loss.backward() # This calculates all the grads wer need!!!!!! Now i understand!! 

In [65]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

# Refactor model


In [66]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [121]:
class Relu(): 
    def __call__(self, inp): 
        self.inp = inp 
        self.out = inp.clamp_min(0.)-0.5
        return self.out
        
    def backward(self): 
        self.inp.g = (self.inp>0).float()*self.out.g 

In [129]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b

        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [123]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [124]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()

In [126]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [127]:
%time loss = model(x_train, y_train)


1
1
CPU times: user 153 ms, sys: 6.12 ms, total: 159 ms
Wall time: 61.4 ms


In [128]:
%time model.backward()


CPU times: user 5.39 s, sys: 2.47 s, total: 7.86 s
Wall time: 7.89 s


In [ ]:
# Module.forward 